# Budget Decider

In [ ]:
import random
import logging
import numpy as np
from tqdm.notebook import trange
import math

In [ ]:
import time
import random
import math
import sys

## Random Search

In [ ]:
def random_search(domain, fitness_function):
  
  # best_cost = sys.maxsize
  best_cost = 0
  for i in range(10000):
    solution = [random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
    cost = fitness_function(solution)
    if cost > best_cost:
      best_cost = cost
      best_solution = solution
  return best_solution

## Genetic Algorithm

In [ ]:
# Mutation Function

def mutation(domain, step, solution):
  gene = random.randint(0, len(domain) - 1)
  mutant = solution
  if random.random() < 0.5:
    if solution[gene] != domain[gene][0]:
      mutant = solution[0:gene] + [solution[gene] - step] + solution[gene + 1:]
  else:
    if solution[gene] != domain[gene][1]:
      mutant = solution[0:gene] + [solution[gene] + step] + solution[gene + 1:]
  return mutant

# CrossOver Function 

def crossover(domain, solution1, solution2):
  gene = random.randint(1, len(domain) - 2)
  return solution1[0:gene] + solution2[gene:]

# Main Genetic Algorithm Function

def genetic(domain, fitness_function, population_size = 100, step = 1,
            probability_mutation = 0.2, elitism = 0.2,
            number_generations = 500, search = False):
  population = []
  for i in range(population_size):
    if search == True:
      solution = random_search(domain, fitness_function)
    else:
      solution = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
    
    population.append(solution)

  number_elitism = int(elitism * population_size)

  for i in range(number_generations):
    costs = [(fitness_function(individual), individual) for individual in population]
    costs.sort()
    ordered_individuals = [individual for (cost, individual) in costs]
    population = ordered_individuals[0:number_elitism]
    while len(population) < population_size:
      if random.random() < probability_mutation:
        m = random.randint(0, number_elitism)
        population.append(mutation(domain, step, ordered_individuals[m]))
      else:
        i1 = random.randint(0, number_elitism)
        i2 = random.randint(0, number_elitism)
        population.append(crossover(domain, ordered_individuals[i1], ordered_individuals[i2]))
  return costs[0][1]

## Hill Climb

In [ ]:
def hill_climb(domain, fitness_function, initial = []):
  count = 0
  
  if len(initial) > 0:
    solution = initial
  else:
    solution = [random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
  
  while True:
    neighbors = []
    for i in range(len(domain)):
      if solution[i] > domain[i][0]:
        if solution[i] != domain[i][1]:
          neighbors.append(solution[0:i] + [solution[i] + 1] + solution[i + 1:])
      if solution[i] < domain[i][1]:
        if solution[i] != domain[i][0]:
          neighbors.append(solution[0:i] + [solution[i] - 1] + solution[i + 1:])

    actual = fitness_function(solution)
    best = actual
    for i in range(len(neighbors)):
      count += 1
      cost = fitness_function(neighbors[i])
      if cost > best:
        best = cost
        solution = neighbors[i]

    if best == actual:
      print('Count: ', count)
      break

  return solution

## Simulated Annealing

In [ ]:
def simulated_anneling(domain, fitness_function, temperature = 50000.0,
                       cooling = 0.95, step = 1, initial = [], search = False):
  count = 0
  if search==True:
    solution = random_search(domain, fitness_function)

  elif len(initial) > 0:
    solution = initial
  else:
    solution = [random.randint(domain[i][0], domain[i][1]) for i in range(len(domain))]
   
  while temperature > 0.1:
    i = random.randint(0, len(domain) - 1)
    direction = random.randint(-step, step)
    temp_solution = solution[:] 
    temp_solution[i] += direction
    if temp_solution[i] < domain[i][0]:
      temp_solution[i] = domain[i][0]
    elif temp_solution[i] > domain[i][1]:
      temp_solution[i] = domain[i][1]

    count += 1
    cost_solution = fitness_function(solution)
    cost_solution_temp = fitness_function(temp_solution)
    probability = pow(math.e, (-cost_solution_temp - cost_solution) / temperature)
    if (cost_solution_temp > cost_solution or random.random() < probability):
      solution = temp_solution

    temperature = temperature * cooling

  print('Count: ', count)
  return solution

## Implementation

In [ ]:
products = [('Rice', 50, 0.8),
            ('Pencils', 20, 0.5),
            ('Chocolates',35, 0.45),
            ('Shampoo', 90, 0.35),
            ('Pasta & Maggi', 70, 0.6),
            ('Butter', 78, 0.44),
            ('Fruits', 130, 0.7)]
# products = [('Rice', 50, 7),
#             ('Pencils', 20, 5),
#             ('Chocolates',35, 4),
#             ('Shampoo', 90, 3),
#             ('Pasta & Maggi', 70, 6),
#             ('Butter', 78, 3),
#             ('Fruits', 130, 5)]

In [ ]:
0.8 + 0.44 + 0.7 + 0.6 + 0.5

3.04

In [ ]:
50 + 130 + 78 + 70 + 20

348

In [ ]:
available_budget = 350

In [ ]:
domain = [(0, 1)] * len(products)
domain

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

In [ ]:
def print_solution(solution):
  for i in range(len(solution)):
    if solution[i] == 1:
      print('%s - %s' % (products[i][0], products[i][1]))

In [ ]:
print_solution([0, 1, 1, 0, 1, 1, 0])

Pencils - 20
Chocolates - 35
Pasta & Maggi - 70
Butter - 78


In [ ]:
def fitness_function(solution):
  value = 0
  sum_cost = 0
  for i in range(len(solution)):
    if solution[i] == 1:
      value += products[i][2]
      sum_cost += products[i][1]
  if sum_cost > available_budget:
    value = 400
  return value

In [ ]:
fitness_function([0, 1, 1, 0, 1, 1, 0])

1.9899999999999998

In [ ]:
solution_random = random_search(domain, fitness_function)
cost = fitness_function(solution_random, verbose=True)
print(cost)
print_solution(solution_random)

3.14 343
3.14
Rice - 50
Pencils - 20
Chocolates - 35
Shampoo - 90
Pasta & Maggi - 70
Butter - 78


In [ ]:
solution_genetic = genetic(domain, fitness_function,search=True)
cost = fitness_function(solution_genetic)
print(cost)
print_solution(solution_genetic)

0


In [ ]:
solution_hill = hill_climb(domain, fitness_function)
cost = fitness_function(solution_hill, verbose=True)
print(cost)
print_solution(solution_hill)

Count:  0
0.85 110
0.85
Pencils - 20
Shampoo - 90


In [ ]:
solution = simulated_anneling(domain, fitness_function,search=True)
cost = fitness_function(solution, verbose=True)
print(cost)
print_solution(solution)

Count:  256
2.8 325
2.8
Rice - 50
Pencils - 20
Chocolates - 35
Shampoo - 90
Fruits - 130


# Neuroevolution

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import copy
from time import time
import sys

In [ ]:
torch.set_grad_enabled(False)

In [ ]:
class Agent(nn.Module):
    '''The brain of the agent'''
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(nn.Linear(2, 16),
                                nn.ReLU(),
                                nn.Linear(16, 1),
                                nn.Sigmoid())
        
    def forward(self, inputs):
        x = self.fc(inputs)
        out = x.round()
        return out

In [ ]:
def initialize_population(pop_size=2):
    '''Randomly initialize a bunch of agents'''
    population = [Agent() for _ in range(pop_size)]
    
    return population

In [ ]:
def evaluate_agent(agent):
    '''Evaluate an agent for the cost'''
    agent.eval()
    solution = []

    total_cost = 0
    total_val = 0

    for _, price, necessity in products:
        x = torch.Tensor([[price/130, necessity]])
        pred = agent(x).squeeze(0).item()
        solution.append(pred)
        if pred == 1:
            total_cost += price
            total_val += necessity
    
    fitness = 0
    if total_cost > available_budget:
        fitness = - (total_cost - available_budget)
    else:
        fitness = total_val

    return fitness

In [ ]:
def evaluate_population(population):
    '''Evaluate the population'''
    pop_fitness = []
    
    for agent in population:
        pop_fitness.append(evaluate_agent(agent))
        
    return pop_fitness

In [ ]:
def mutate(parent_agent, mutation_power=0.02):
    '''Creates a mutated copy of the parent agent by adding a weighted gaussian noise to the params'''
    child_agent = copy.deepcopy(parent_agent)
    
    for param in child_agent.parameters():
        param.data = param.data + (torch.randn(param.shape) * mutation_power)
        
    return child_agent

In [ ]:
def repopulate(top_agents, pop_size, mutation_power):
    '''Repopulate the population from the top agents by mutation'''
    new_population = []
    
    n = 0
    while(n < pop_size):
        for parent in top_agents:
            child = mutate(parent, mutation_power)
            new_population.append(child)
            n += 1
            
    return new_population[:pop_size - 1]

In [ ]:
def evolve(generations=10, 
           pop_size=100, 
           topK=20,
           mutation_power=0.02):
    '''Start the process of evolution'''
    
    global TRAINED_AGENT
    
    population = initialize_population(pop_size)
    global_best = {}
    
    for g in range(generations):
        
        # Evaluate the population
        pop_fitness = evaluate_population(population)
        mean_pop_reward = np.array(pop_fitness).mean()
        
        # Rank the agents in descending order
        topK_idx = np.argsort(pop_fitness)[::-1][:topK]
        topK_agents = [population[i] for i in topK_idx]
        
        # Get Best Agent
        best_agent = population[topK_idx[0]]
        best_reward = pop_fitness[topK_idx[0]]
        
        # Check with global best
        if g == 0:
            global_best['reward'] = best_reward
            global_best['agent'] = best_agent
        else:
            if best_reward >= global_best['reward']:
                global_best['reward'] = best_reward
                global_best['agent'] = best_agent
                
        print('Generation', g)
        print('Mean Reward of Population', mean_pop_reward)
        print('Best Agent Reward (mean)', best_reward)
        print('Global Best Reward (mean)', global_best['reward'], '\n')
        
        # Mutate and Repopulate
        new_population = repopulate(topK_agents, pop_size, mutation_power)
        # take the best agent of generation forward without cloning as well
        new_population.append(best_agent)
        
        population = new_population
        
        TRAINED_AGENT = global_best

In [ ]:
def get_solution(agent, products, verbose=True):
    print_sol = []
    solution = []
    total_cost = 0
    total_val = 0

    for product, price, necessity in products:
        x = torch.Tensor([[price/130, necessity]])
        pred = agent(x).squeeze(0).item()
        solution.append(pred)
        if pred == 1:
            total_cost += price
            total_val += necessity
            print_sol.append(f"{product}: Rs {price}")

    if verbose:
        print(f'Total Cost: Rs {total_cost}')
        print(f'Necessity Value: {total_val}\n')
        print('\n'.join(print_sol))

    return solution, total_cost, total_val

In [ ]:
products = [('Rice', 50, 0.8),
            ('Pencils', 20, 0.5),
            ('Chocolates',35, 0.45),
            ('Shampoo', 90, 0.35),
            ('Pasta & Maggi', 70, 0.6),
            ('Butter', 78, 0.44),
            ('Fruits', 130, 0.7)]

available_budget = 350

In [ ]:
TRAINED_AGENT = {}

In [ ]:
evolve(generations=30,
       pop_size=20, 
       topK=10,
       mutation_power=0.02)

In [ ]:
solution, cost, value = get_solution(TRAINED_AGENT['agent'], products)